In [198]:
import requests
import urllib
import pandas as pd

In [199]:
session = requests.Session()
session.headers.update({'Accept': 'application/json'})


In [200]:
base_url = "https://unmasked.nation.africa/"

response = session.get("https://unmasked.nation.africa/@search?fullobjects=1&portal_type=Membership")

In [201]:
# Points: who,with
# Links: who,bio,category,photo,size

In [202]:
memberships = response.json()['items']

Ideally we should use networkx to build and extract relationships, this is a hack

# Points

In [203]:
key_person_links = []
    
#Key person

key_person = "Uhuru Kenyatta"
for member in memberships:
    
    if member['person']['title']== key_person:
        key_person_links.append({"Who" : member['person']['title'],
                                 "who_id" : member['person']['@id'],
                                 "Associated With" : member['organization']['title'],
                                 "associated_id" : member['organization']['@id'],})

In [204]:
#For associated persons

associated_person_links = []

for link in key_person_links:
    if link.get('Associated With'):
        for member in memberships:
            if member['organization']['title']==link['Associated With']:
                if member['person']['title'] != key_person:
                    associated_person_links.append({"Who" : member['person']['title'],
                                                    "who_id" : member['person']['@id'],
                                                    "Associated With" : member['organization']['title'],
                                                    "associated_id" : member['organization']['@id'],})
            

In [205]:
points = key_person_links + associated_person_links

In [206]:
points

[{'Who': 'Uhuru Kenyatta',
  'who_id': 'https://unmasked.nation.africa/people/uhuru-kenyatta',
  'Associated With': 'Jubilee Party',
  'associated_id': 'https://unmasked.nation.africa/organisations/jubilee-party'},
 {'Who': 'David Murathe',
  'who_id': 'https://unmasked.nation.africa/people/david-murathe',
  'Associated With': 'Jubilee Party',
  'associated_id': 'https://unmasked.nation.africa/organisations/jubilee-party'}]

In [207]:
df_points = pd.DataFrame(points)
df_points

,Who,who_id,Associated With,associated_id
0,Uhuru Kenyatta,https://unmasked.nation.africa/people/uhuru-ke...,Jubilee Party,https://unmasked.nation.africa/organisations/j...
1,David Murathe,https://unmasked.nation.africa/people/david-mu...,Jubilee Party,https://unmasked.nation.africa/organisations/j...


# Links

Links: who,bio,category,photo,size

In [208]:

# Individuals

individuals = []

for index, person in df_points.drop_duplicates(subset=['Who']).iterrows():
    # get details from API
    
    url = person['who_id']+'?fullobjects=1'
    response = session.get(url)
    individual = response.json()
    if individual['image']:
        image_url = individual['image']['download'] 
    else:
        image_url = None
    individuals.append({'Who': person['Who'],
                        'Bio': individual['summary'],
                        'Category': 'Individual',
                        'Photo': image_url,
                       })

organizations = [] 

for index, org in df_points.drop_duplicates(subset=['Associated With']).iterrows():
    # get details from API
    
    url = org['associated_id']+'?fullobjects=1'
    response = session.get(url)
    company = response.json()
    if company['image']:
        image_url = company['image']['download'] 
    else:
        image_url = None
    organizations.append({'Who': org['Associated With'],
                        'Bio': company['description'],
                        'Category': 'Company',
                        'Photo': image_url,
                       })       




In [209]:
individuals

[{'Who': 'Uhuru Kenyatta',
  'Bio': 'Uhuru Muigai Kenyatta (born 26 October 1961) is a Kenyan politician, businessman, and the fourth and current President of the Republic of Kenya.',
  'Category': 'Individual',
  'Photo': 'https://unmasked.nation.africa/people/uhuru-kenyatta/@@images/5b140073-8bc1-43f2-a451-c3cb7c68e5d0.png'},
 {'Who': 'David Murathe',
  'Bio': 'David Murathe is a politician, vice-president of Jubilee Party',
  'Category': 'Individual',
  'Photo': None}]

In [210]:
organizations

[{'Who': 'Jubilee Party',
  'Bio': 'The Jubilee Party of Kenya is the ruling political party of the Republic of Kenya. The party was founded on 8 September 2016, following the merger of 11 smaller parties.',
  'Category': 'Company',
  'Photo': 'https://unmasked.nation.africa/organisations/jubilee-party/@@images/fd00ae3c-7416-42f9-aacb-7c0ac05928eb.png'}]

In [211]:
links = individuals + organizations

In [212]:
df_links = pd.DataFrame(links)
df_links

,Who,Bio,Category,Photo
0,Uhuru Kenyatta,Uhuru Muigai Kenyatta (born 26 October 1961) i...,Individual,https://unmasked.nation.africa/people/uhuru-ke...
1,David Murathe,"David Murathe is a politician, vice-president ...",Individual,None
2,Jubilee Party,The Jubilee Party of Kenya is the ruling polit...,Company,https://unmasked.nation.africa/organisations/j...


In [215]:
df_points.to_csv('points.csv',index=False)
df_links.to_csv('links.csv',index=False)